In [1]:
# from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index import download_loader

CJKPDFReader = download_loader("CJKPDFReader")
loader = CJKPDFReader()
documents = loader.load_data(file="pdf_data/data_1.pdf")
print(len(documents))

1


In [2]:
with open("./pdf_to_text/pdf_to_text_1.txt",encoding="utf-8",mode="w") as f:
    f.write(documents[0].text) 

In [3]:
from qdrant_client import QdrantClient,models
from qdrant_client.http.models import PointStruct

connection = QdrantClient(host='qdrant', port=6333)
print("Connection successful:", connection)

connection.create_collection(
    collection_name="test_rag1",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)
info = connection.get_collection(collection_name="test_rag1")

Connection successful: <qdrant_client.qdrant_client.QdrantClient object at 0x7f77c0cad590>


In [4]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
import os
import uuid
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",chunk_size=1000,chunk_overlap=200,length_function=len)
    chunks = text_splitter.split_text(text)
    return chunks

def get_embedding(text_chunks, model_id="text-embedding-ada-002"):
    points = []
    for idx, chunk in enumerate(text_chunks):
        response = openai.Embedding.create(
            input=chunk,
            model=model_id
        )
        embeddings = response['data'][0]['embedding']
        point_id = str(uuid.uuid4())  # Generate a unique ID for the point
        points.append(PointStruct(id=point_id, vector=embeddings, payload={"text": chunk}))

    return points

def insert_data(get_points):
    operation_info = connection.upsert(
        collection_name="test_rag1",
        wait=True,
        points=get_points
    )

def create_answer_with_context(query):
    response = openai.Embedding.create(
        input=query,

        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']
    search_result = connection.search(
        collection_name="test_rag1",
        query_vector=embeddings,
        limit=3
    )
    print("Question: " ,query,'\n')
    print("Searching.......\n")
    prompt=""
    for result in search_result:
        prompt += result.payload['text']
    concatenated_string = " ".join([prompt,query])
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": concatenated_string}
        ]
        )
    return completion.choices[0].message.content

def main():
    get_raw_text=documents[0].text
    chunks=get_text_chunks(get_raw_text)
    vectors=get_embedding(chunks)
    
    insert_data(vectors)
    question="提案手法の概要を説明してください"
    answer=create_answer_with_context(question)
    print("Answer : ",answer,"\n")
    print("searching completed")


In [5]:
main()

Question:  提案手法の概要を説明してください 

Searching.......

Answer :  本研究では、機械学習における異常検知モデルの学習データセットを圧縮するための手法を提案しています。提案手法の概要は以下の通りです。

1. 位置合わせ: データセット内の各画像のオブジェクトの位置を合わせるために、マスク画像を使用します。これにより、オブジェクトが干渉しない範囲の背景を削除することができます。

2. 背景削除: マスク画像を使用して、オブジェクト以外の背景を削除します。この処理により、データ間の類似性を高めることができます。

3. 動画ファイルへの変換: 加工された画像データセットをHEVCを使用して動画ファイルに変換します。HEVCによる変換は、画像データセットのサイズを大幅に縮小することができます。

4. 学習とテストに向けた前処理: 学習時には動画ファイルを画像ファイルに復元し、従来の方法で学習を行います。テスト用データも同様に加工して精度の維持を図ります。

提案手法の評価では、異常検知モデルの精度を確認し、データサイズの縮小効果や位置合わせ・背景削除の効果を評価します。比較手法として従来の手法やHEVCの圧縮パラメータを変更した手法などを用いて評価を行います。評価結果から、提案手法がモデルの精度を落とさずにデータサイズを縮小できることを確認することが目的です。 

searching completed
